In [1]:
from datetime import datetime
import re
import pandas as pd
from datetime import datetime, timedelta
from unidecode import unidecode
df={'Origin':['BOGOTA','VILLAVICENCIO','BUCARAMANGA'],
    'Destination':['BOGOTA','VILLAVICENCIO','BUCARAMANGA'],
    'Cupo':['AAV837','DRZ986','FVN345'],
    'ESTADO':['MANTENIMIENTO','EN SERVICIO','MANTENIMIENTO'],
    'Fecha Incio':['09/13/2023','09/12/2023','09/10/2023'],
    'Fecha Finalizacion':['09/15/2023','09/12/2023','']   
    }

Mantenimiento=pd.read_excel('Mantenimiento.xlsx',header=0)
df_formato_guia = pd.read_excel('ACT_Formato_guia_control_renta.xlsx',header=0)
df_entrapetrol=pd.read_excel
columnas=['Origin','Destination','Cupo', 'ESTADO','Fecha Inicio','Fecha Finalizacion']
#print(f"{Mantenimiento}\n")
print(df_formato_guia.loc[df_formato_guia["ESTADO"]=="MANTENIMIENTO",columnas])
df=pd.DataFrame(df)

                               Origin     Destination    Cupo         ESTADO  \
0                            TIBASOSA          BOGOTA  FVN341  MANTENIMIENTO   
3                     BARRANCABERMEJA  PUERTO WILCHES  FVQ473  MANTENIMIENTO   
4                          VALLEDUPAR     SANTA MARTA  FVQ475  MANTENIMIENTO   
6                               YOPAL            COTA  FVQ491  MANTENIMIENTO   
11                              YOPAL          BOGOTA  FVR348  MANTENIMIENTO   
13  HOTEL NH COLLECTION EN CALLE 100.          BOGOTA  FVS295  MANTENIMIENTO   
29                           SOGAMOSO        MOSQUERA  FVR346  MANTENIMIENTO   

   Fecha Inicio Fecha Finalizacion  
0    2024-01-04                NaT  
3    2024-01-13                NaT  
4    2023-10-17                NaT  
6    2023-12-18                NaT  
11   2023-12-18                NaT  
13   2023-12-18                NaT  
29   2024-01-13                NaT  


In [2]:
# Fecha actual
hoy = datetime.now().date()
# Itera sobre las filas de df_formato_guia
for index, row in df_formato_guia.iterrows():
    if row['ESTADO'] == "MANTENIMIENTO":
        # Verifica si el Cupo existe en Mantenimiento
        if row['Cupo'] not in Mantenimiento['Cupo'].values:
            # Crea una nueva fila para agregar a Mantenimiento
            fecha_inicio = pd.to_datetime(row['Fecha Inicio']).date()  # Convert to datetime and extract date
            dias = (hoy - fecha_inicio).days
            dias = dias + 1
            nueva_fila = pd.DataFrame({'Cupo': [row['Cupo']], 'Inicio': [row['Fecha Inicio']],
                                       'Finalizo': [None], 'Dias': [dias], 'Observacion': [row['Observaciones']]})
            # Concatenar el nuevo DataFrame con Mantenimiento
            Mantenimiento = pd.concat([Mantenimiento, nueva_fila], ignore_index=True)
        else:
            # Busca la fila existente en Mantenimiento
            index_mantenimiento = Mantenimiento[Mantenimiento['Cupo'] == row['Cupo']].index[0]
            fecha_inicio_actual = pd.to_datetime(row['Fecha Inicio']).date()  # Convert to datetime and extract date
            fecha_inicio_mantenimiento = pd.to_datetime(Mantenimiento.loc[index_mantenimiento, 'Inicio']).date()  # Convert to datetime and extract date
            dias = (hoy - fecha_inicio_mantenimiento).days
            Mantenimiento.at[index_mantenimiento, 'Dias'] = dias + 1
            Mantenimiento.at[index_mantenimiento, 'Inicio'] = row['Fecha Inicio']  # Actualiza la columna 'Inicio'

    elif row['ESTADO'] != "MANTENIMIENTO":
        # Verifica si el Cupo existe en Mantenimiento y si no tiene fecha de finalización
        if row['Cupo'] in Mantenimiento['Cupo'].values and pd.isna(Mantenimiento[Mantenimiento['Cupo'] == row['Cupo']]['Finalizo'].values[0]):
            # Actualiza la fecha de finalización en Mantenimiento
            index_mantenimiento = Mantenimiento[Mantenimiento['Cupo'] == row['Cupo']].index[0]
            fecha_inicio_actual = pd.to_datetime(Mantenimiento.loc[Mantenimiento['Cupo'] == row['Cupo'], 'Inicio'].iloc[0]).date()  # Convert to datetime and extract date
            if row['ESTADO'] == 'DISPONIBLE':
                Mantenimiento.at[index_mantenimiento, 'Finalizo'] = hoy
            Mantenimiento.loc[index_mantenimiento, 'Finalizo'] = fecha_inicio_actual.strftime("%m/%d/%Y %H:%M:%S")
            dias = (hoy - fecha_inicio_actual).days
            Mantenimiento.at[index_mantenimiento, 'Dias'] = dias + 1

column_order = ['Cupo', 'Inicio', 'Finalizo', 'Dias', 'Observacion']
Mantenimiento = Mantenimiento[column_order]
Mantenimiento.to_excel('Mantenimiento.xlsx', index=False)
# Imprime el DataFrame Mantenimiento actualizado
print(Mantenimiento)


     Cupo               Inicio             Finalizo  Dias  \
0  FVQ475  2023-10-17 00:00:00                  NaN    91   
1  FVQ491  2023-12-18 00:00:00                  NaN    29   
2  FVR348  2023-12-18 00:00:00                  NaN    29   
3  FVS295  2023-12-18 00:00:00                  NaN    29   
4  LHR715  01/11/2024 06:00:00  01/11/2024 00:00:00     5   
5  FVN341  2024-01-04 00:00:00                  NaN    12   
6  FVS298  2023-12-30 00:00:00  12/30/2023 00:00:00    17   
7  FVQ473  2024-01-13 00:00:00                 None     3   
8  FVR346  2024-01-13 00:00:00                 None     3   

                                    Observacion  
0  ingresa a mantenimiento por fallas mecánicas  
1                                           NaN  
2                                           NaN  
3                                           NaN  
4                                           NaN  
5                                           NaN  
6                      Esperado para ci